In [3]:
from faker import Faker
# Conexion
import psycopg2
import uuid

# Configuración de la conexion
conexion = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="contraseña"
)

# Consulta para obtener las tablas de un esquema
esquema = 'recruitment'
tablas_db = []
consulta = f"""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = '{esquema}' AND table_type = 'BASE TABLE';
"""

# Ejecutar la consulta
with conexion.cursor() as cursor:
    cursor.execute(consulta)
    tablas = cursor.fetchall()
    tablas_db = tablas

In [4]:

# def recorrer_tabla(tabla):
#     # Consulta para obtener las columnas de una tabla
#     consulta = f"""
#     SELECT column_name, data_type
#     FROM information_schema.columns
#     WHERE table_schema = %s AND table_name = %s;
#     """

#     # Ejecutar la consulta
#     with conexion.cursor() as cursor:
#         cursor.execute(consulta, (esquema, tabla))
#         columnas = cursor.fetchall()

#     # Cerrar la conexión

#     print(tabla)
#     # Mostrar las columnas
#     for columna in columnas:
#         print(f"Columna: {columna[0]}, Tipo de dato: {columna[1]}")

In [5]:
# for tabla in tablas_db:
#     recorrer_tabla(tabla)

In [6]:
# conexion.close()

In [7]:
# for tabla in tablas_db:
#     print(tabla[0])

In [8]:
# Instancia Faker
fake = Faker()
import random

In [9]:
def generar_datos_applicant_profile(num_registros):
    datos = []
    for _ in range(num_registros):
        nombre = fake.name()
        email = fake.email()
        id_usuario = str(uuid.uuid4())
        
        # Aquí puedes añadir otras columnas según el diseño de tu tabla
        datos.append((nombre, email, id_usuario))
    return datos

In [10]:
registros = generar_datos_applicant_profile(5000)

In [11]:
registros

[('Tonya Dixon', 'mjones@example.org', '84d2f64d-2bdd-4091-86dd-f9dc39b397d1'),
 ('James Pierce',
  'brandonflores@example.com',
  '24ecdfbf-7344-42b9-997b-b18c078e71d4'),
 ('Lori Carr', 'hannah22@example.org', '226f6c89-81a6-4547-b222-9ba2782ecc03'),
 ('Gabriela Phillips',
  'christopher89@example.org',
  '19907e69-b46c-4a93-b7e6-55504640285e'),
 ('Michael Johnson',
  'howellmichelle@example.org',
  'e021cc51-b29f-4149-9ecb-69fbd27c5d7a'),
 ('Sarah Lewis',
  'edward91@example.com',
  '818ba39d-b23e-4dfa-8d33-475fbd79bf8d'),
 ('Breanna Fox',
  'brandymartin@example.com',
  '714c0181-5835-4ecb-855c-9ee8baa794c6'),
 ('Carla Sullivan',
  'gonzalezwilliam@example.org',
  '8edada9b-9ac5-49db-a998-8f63cc08820a'),
 ('Dale Fitzgerald',
  'bishopmichael@example.com',
  'b690722a-5d90-44d9-861b-93c136731349'),
 ('Roger Daniel',
  'ariley@example.net',
  '30609ea9-a70b-470b-86f0-f853f967adb1'),
 ('Brooke Holloway',
  'jhubbard@example.com',
  '3867551d-8cba-4787-a999-d707a93e98e3'),
 ('John Powel

In [12]:
# Insertar los datos generados en la base de datos
def insertar_datos_applicant_profile(datos):
    with conexion.cursor() as cursor:
        # Consulta SQL para insertar registros
        
        consulta = """
        INSERT INTO recruitment.user (name, email, user_id)
        VALUES (%s, %s, %s)
        """
        # Insertar múltiples registros
        cursor.executemany(consulta, datos)
        conexion.commit()  # Confirmar la transacción

In [13]:
insertar_datos_applicant_profile(registros)

In [14]:
import psycopg2

def insertar_user_ids_no_existentes():
    try:
        # Conectar a la base de datos
        with psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="contraseña"
        ) as conexion:
            with conexion.cursor() as cursor:
                # Leer los 'user_id' existentes en la tabla 'user'
                consulta_user_ids = """
                SELECT user_id FROM recruitment.user
                """
                cursor.execute(consulta_user_ids)
                user_ids_existentes = set(row[0] for row in cursor.fetchall())  # Almacenar los user_id en un set

                # Leer los 'user_id' que ya existen en la tabla 'applicant_profile'
                consulta_applicant_profile = """
                SELECT user_id FROM recruitment.applicant_profile
                """
                cursor.execute(consulta_applicant_profile)
                user_ids_en_profile = set(row[0] for row in cursor.fetchall())  # Almacenar los user_id en applicant_profile

                # Identificar los 'user_id' que están en 'user' pero no en 'applicant_profile'
                user_ids_a_insertar = user_ids_existentes - user_ids_en_profile

                if user_ids_a_insertar:
                    # Insertar los 'user_id' que faltan en la tabla 'applicant_profile'
                    consulta_insertar = """
                    INSERT INTO recruitment.applicant_profile (user_id)
                    VALUES (%s)
                    """
                    cursor.executemany(consulta_insertar, [(user_id,) for user_id in user_ids_a_insertar])
                    conexion.commit()  # Confirmar la transacción

                    print(f"{len(user_ids_a_insertar)} user_ids insertados correctamente en 'applicant_profile'.")
                else:
                    print("No hay nuevos user_ids para insertar en 'applicant_profile'.")
                    return True

    except Exception as e:
        # Imprimir el error en caso de que ocurra una excepción
        print(f"Ocurrió un error: {e}")
        # Si ocurre un error, hacer rollback de cualquier cambio no confirmado
        if 'conexion' in locals():
            conexion.rollback()


In [15]:
while (True):
    if(insertar_user_ids_no_existentes()):
        break

5000 user_ids insertados correctamente en 'applicant_profile'.
No hay nuevos user_ids para insertar en 'applicant_profile'.


In [16]:
import psycopg2
import random

def asignar_skills_random_a_applicants():
    try:
        # Conectar a la base de datos
        conexion = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="contraseña"
        )

        with conexion.cursor() as cursor:
            # Obtener todos los applicant_profile_id y sus skills asignados
            consulta_applicant_skills = """
            SELECT applicant_profile_id, COUNT(skill_id) 
            FROM recruitment.applicantprofile_skill
            GROUP BY applicant_profile_id
            """
            cursor.execute(consulta_applicant_skills)
            skills_por_applicant = {row[0]: row[1] for row in cursor.fetchall()}

            # Obtener todos los applicant_profile_id de la tabla 'applicant_profile'
            consulta_applicants = """
            SELECT id FROM recruitment.applicant_profile
            """
            cursor.execute(consulta_applicants)
            applicant_ids = [row[0] for row in cursor.fetchall()]

            # Obtener los skill_id válidos desde la tabla 'skills'
            consulta_skills = """
            SELECT id FROM recruitment.skills
            """
            cursor.execute(consulta_skills)
            skill_id_validos = [row[0] for row in cursor.fetchall()]

            if not skill_id_validos:
                print("No hay skills válidos disponibles en la tabla 'skills'.")
                return

            # Crear combinaciones de applicant_profile_id con nuevos skill_id aleatorios
            registros_a_insertar = []
            for applicant_id in applicant_ids:
                # Verificar si ya tiene 7 o más skills asignados
                skills_actuales = skills_por_applicant.get(applicant_id, 0)
                if skills_actuales >= 8:
                    continue

                # Determinar cuántos skills adicionales se necesitan
                skills_faltantes = 8 - skills_actuales
                if len(skill_id_validos) < skills_faltantes:
                    print(f"No hay suficientes skills válidos para el applicant {applicant_id}.")
                    continue
                    
                # Seleccionar skill_id únicos al azar que aún no estén asignados
                skill_ids_random = random.sample(skill_id_validos, skills_faltantes)
                registros_a_insertar.extend([(applicant_id, skill_id) for skill_id in skill_ids_random])

            # Insertar las combinaciones en la tabla 'applicantprofile_skill'
            if registros_a_insertar:
                consulta_insertar = """
                INSERT INTO recruitment.applicantprofile_skill (applicant_profile_id, skill_id)
                VALUES (%s, %s)
                """
                cursor.executemany(consulta_insertar, registros_a_insertar)
                conexion.commit()
                print(f"Se asignaron nuevos skills a los applicants.")
            else:
                print("No se encontraron applicants que requieran más skills.")

    except Exception as e:
        print(f"Ocurrió un error: {e}")
        if conexion:
            conexion.rollback()
    finally:
        if conexion:
            conexion.close()

# Llamar a la función
asignar_skills_random_a_applicants()

Se asignaron nuevos skills a los applicants.


In [17]:
import random
from datetime import datetime, timedelta
import psycopg2

# Configuración de conexión
conexion = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="contraseña"
        )

cursor = conexion.cursor()

roles_tecnologicos = [
    "Desarrollador Backend",
    "Desarrollador Frontend",
    "Desarrollador Full Stack",
    "Ingeniero de Software",
    "Administrador de Bases de Datos",
    "Arquitecto de Software",
    "Analista de Datos",
    "Científico de Datos",
    "Data Engineer",
    "Ingeniero de Machine Learning",
    "Desarrollador de Aplicaciones Móviles",
    "Administrador de Sistemas",
    "DevOps Engineer",
    "Ingeniero en Seguridad Informática",
    "Especialista en Cloud",
    "Administrador de Redes",
    "Consultor de Tecnología",
    "Gestor de Proyectos de TI",
    "Desarrollador de Videojuegos",
    "Ingeniero de IA (Inteligencia Artificial)",
    "Ingeniero de datos"
]

# Función para generar un registro aleatorio
def generar_registro(id_job):
    active = random.choice([True, False])
    endtime = (datetime.now() + timedelta(hours=random.randint(1, 8))).time() if random.choice([True, False]) else None
    language_id = random.randint(1, 10)
    levelofexperience_id = random.randint(1, 5)
    maxsalary = round(random.uniform(50000, 100000), 2)
    minsalary = round(random.uniform(30000, 50000), 2)
    openpositions = random.randint(1, 10)
    profile_type_id = random.randint(1, 16)
    salary = round(random.uniform(50000, 120000), 2)
    starttime = (datetime.now() - timedelta(hours=random.randint(1, 8))).time() if random.choice([True, False]) else None
    verified_profiles = random.choice([True, False])
    visibility_name_company = random.choice([True, False])
    company_id = random.randint(1, 11)
    country_id = random.randint(1, 11)
    createdat = datetime.now() - timedelta(days=random.randint(1, 365))
    updateat = datetime.now() - timedelta(days=random.randint(1, 30))
    description = f"Descripción genérica {id_job}"
    description_functions = f"Funciones del puesto {id_job}"
    description_functions_additional_requirements = f"Requisitos adicionales {id_job}"
    name = random.choice(roles_tecnologicos)
    workmode = random.choice(["PRESENCIAL", "REMOTO","HIBRIDO"])

    return (
        active, endtime, language_id, levelofexperience_id, maxsalary, minsalary,
        openpositions, profile_type_id, salary, starttime, verified_profiles,
        visibility_name_company, company_id, country_id, createdat, updateat,
        description, description_functions, description_functions_additional_requirements,
        name, workmode
    )

# SQL para insertar registros
sql_insert = """
INSERT INTO recruitment.job (
    active, endtime, language_id, levelofexperience_id, maxsalary, minsalary,
    openpositions, profile_type_id, salary, starttime, verified_profiles,
    visibility_name_company, company_id, country_id, createdat, updateat,
    description, description_functions, description_functions_additional_requirements,
    name, workmode
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
);
"""

# Generar e insertar 100 registros
registros = [generar_registro(i) for i in range(1, 5000)]
cursor.executemany(sql_insert, registros)

# Confirmar los cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()

print("¡Registros insertados con éxito!")


¡Registros insertados con éxito!


In [18]:
import random
from datetime import datetime, timedelta
import psycopg2

# Configuración de conexión
try:
    conexion = psycopg2.connect(
        host="localhost",
        database="postgres",
        user="postgres",
        password="contraseña"
    )

    cursor = conexion.cursor()

    # Obtener todos los user_id de la tabla user en orden
    cursor.execute("SELECT user_id FROM recruitment.user ORDER BY user_id")
    user_ids = [row[0] for row in cursor.fetchall()]

    # Obtener todos los job_id de la tabla job en orden
    cursor.execute("SELECT id FROM recruitment.job ORDER BY id")
    job_ids = [row[0] for row in cursor.fetchall()]

    # Verificar si hay suficientes trabajos para los usuarios
    if len(user_ids) > len(job_ids):
        print(len(user_ids), len(job_ids))
        print("Error: No hay suficientes trabajos para todos los usuarios.")
    else:
        # Función para generar un registro
        def generar_registro(user_id, job_id):
            return (job_id, user_id)

        # Generar registros para cada usuario y asignarles un job_id en orden
        registros = []

        # Verificar qué combinaciones de user_id y job_id ya existen en la tabla application
        for i in range(len(user_ids)):
            user_id = user_ids[i]
            job_id = job_ids[i]

            # Verificar si ya existe una combinación de user_id y job_id en la tabla application
            cursor.execute("""
                SELECT 1 FROM recruitment.application 
                WHERE job_id = %s AND user_id = %s
            """, (job_id, user_id))
            if cursor.fetchone() is None:  # Si no existe, agregamos el registro a la lista
                registros.append(generar_registro(user_id, job_id))

        # SQL para insertar los registros en la tabla application (sin incluir el id)
        sql_insert = """
        INSERT INTO recruitment.application (
            job_id, user_id
        ) VALUES (
            %s, %s
        );
        """

        # Insertar los registros en la base de datos si hay registros pendientes
        if registros:
            batch_size = 1000  # Definir el tamaño del lote
            for i in range(0, len(registros), batch_size):
                cursor.executemany(sql_insert, registros[i:i+batch_size])

            # Confirmar los cambios y cerrar la conexión
            conexion.commit()
            print(f"¡{len(registros)} registros insertados con éxito!")
        else:
            print("No se encontraron registros nuevos para insertar.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

finally:
    # Cerrar la conexión
    if cursor:
        cursor.close()
    if conexion:
        conexion.close()


¡21271 registros insertados con éxito!


In [19]:
import psycopg2
import random
from datetime import date, timedelta

def generar_inserts_workexperience():
    # Lista de descripciones en español
    descripciones = [
    "Implementó una arquitectura de microservicios con Java y Spring Boot en un entorno de integración continua (CI/CD).",
    "Optimizó consultas SQL, logrando reducir el tiempo de respuesta de la base de datos en un 40%.",
    "Desarrolló una aplicación móvil nativa utilizando Swift y Kotlin, mejorando la experiencia de usuario.",
    "Diseñó e implementó pipelines de datos con Python y Apache Kafka para procesar información en tiempo real.",
    "Lideró la migración de sistemas monolíticos a una arquitectura basada en contenedores con Docker y Kubernetes.",
    "Automatizó pruebas de software utilizando Selenium y Pytest, incrementando la cobertura de pruebas en un 35%.",
    "Desarrolló APIs RESTful robustas y escalables con Node.js y Express, soportando más de 10,000 solicitudes por minuto.",
    "Colaboró en la implementación de un sistema de autenticación basado en OAuth2 y OpenID Connect.",
    "Participó en el diseño de bases de datos NoSQL con MongoDB para manejar grandes volúmenes de datos no estructurados.",
    "Integró herramientas de monitoreo como Prometheus y Grafana para rastrear métricas del sistema en tiempo real.",
    "Desplegó aplicaciones empresariales en la nube utilizando AWS, aprovechando servicios como EC2 y Lambda.",
    "Contribuyó a un proyecto de aprendizaje automático para predecir tendencias del mercado utilizando TensorFlow.",
    "Refactorizó código legado para mejorar la mantenibilidad y el rendimiento, reduciendo errores críticos en un 20%.",
    "Implementó un sistema de notificaciones push en tiempo real utilizando WebSockets y Redis.",
    "Configuró y gestionó pipelines CI/CD en Jenkins para garantizar despliegues automáticos y sin errores.",
    "Automatizó la generación de reportes financieros utilizando Power BI y SQL Server.",
    "Desarrolló un chatbot basado en inteligencia artificial para mejorar la atención al cliente.",
    "Implementó seguridad avanzada en aplicaciones web mediante la adopción de prácticas de OWASP.",
    "Diseñó interfaces de usuario intuitivas y responsivas con React y Material-UI.",
    "Lideró un equipo ágil de desarrollo en la entrega de un proyecto antes del plazo estipulado.",
    "Implementó estrategias de caché con Redis para reducir la latencia de las aplicaciones en un 50%.",
    "Desarrolló sistemas de análisis de datos utilizando pandas y Jupyter Notebook para informes dinámicos.",
    "Configuró clústeres Hadoop para procesamiento masivo de datos en un entorno empresarial.",
    "Automatizó procesos manuales mediante el desarrollo de scripts en Bash y Python.",
    "Participó en el diseño de algoritmos de recomendación para una plataforma de e-commerce.",
    "Rediseñó el frontend de un sistema heredado con Angular, mejorando la velocidad de carga en un 30%.",
    "Aseguró la escalabilidad del sistema mediante la implementación de un balanceador de carga con NGINX.",
    "Migró bases de datos desde Oracle a PostgreSQL, reduciendo costos operativos significativamente.",
    "Lideró una auditoría de código para garantizar el cumplimiento de estándares de calidad y seguridad.",
    "Desarrolló integraciones con APIs externas para sincronizar datos entre múltiples plataformas.",
    "Implementó autenticación multifactor en una aplicación web para mejorar la seguridad.",
    "Creó dashboards personalizados para monitoreo de KPIs críticos utilizando Tableau.",
    "Gestionó la infraestructura como código utilizando Terraform en un entorno multi-cloud.",
    "Diseñó estrategias de respaldo y recuperación de datos para sistemas críticos en producción.",
    "Construyó una herramienta de automatización que redujo el tiempo de configuración en un 60%.",
    "Colaboró en un proyecto de IoT para recopilar y analizar datos de sensores remotos.",
    "Implementó un sistema de procesamiento batch utilizando Apache Spark para grandes volúmenes de datos.",
    "Desarrolló una biblioteca reusable para mejorar la productividad en proyectos internos.",
    "Mejoró la seguridad de una aplicación mediante la implementación de cifrado de extremo a extremo.",
    ]


    try:
        # Conexión a la base de datos
        conexion = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="contraseña"
        )

        with conexion.cursor() as cursor:
            # Obtener los IDs de todos los applicant_profiles
            consulta_applicant_profiles = "SELECT id FROM recruitment.applicant_profile"
            cursor.execute(consulta_applicant_profiles)
            applicant_profile_ids = [row[0] for row in cursor.fetchall()]

            # Obtener los applicant_profile_id ya existentes en la tabla recruitment.offer
            consulta_ofertas_existentes = "SELECT applicantprofile_id FROM recruitment.workexperience"
            cursor.execute(consulta_ofertas_existentes)
            ofertas_existentes = {row[0] for row in cursor.fetchall()}

            # Generar inserts solo para los applicant_profiles que no tienen oferta
            inserts = []
            for applicant_id in applicant_profile_ids:
                if applicant_id in ofertas_existentes:
                    continue  # Si ya tiene una oferta, se omite

                # Generar datos aleatorios
                end_date = date.today() + timedelta(days=random.randint(30, 365))
                initiate_date = date.today() - timedelta(days=random.randint(0, 365))
                country_id = random.randint(1, 10)  # Ejemplo de rango para country_id
                company = f"Compañía {random.randint(1, 100)}"
                description = random.choice(descripciones)
                position = f"Posición {random.randint(1, 50)}"

                inserts.append(
                    (applicant_id, initiate_date, end_date, country_id, company, description, position)
                )

            # Insertar las nuevas ofertas en la tabla
            if inserts:
                consulta_insert = """
                INSERT INTO recruitment.workexperience (
                    applicantprofile_id, initiate_date, end_date, country_id, company, description, position
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                """
                cursor.executemany(consulta_insert, inserts)
                conexion.commit()
                print(f"Se generaron {len(inserts)} nuevos workexp en la tabla recruitment.workexperience.")
            else:
                print("No se encontraron applicant_profiles sin ofertas existentes.")

    except Exception as e:
        print(f"Ocurrió un error: {e}")
        if conexion:
            conexion.rollback()
    finally:
        if conexion:
            conexion.close()

# Llamar a la función
generar_inserts_workexperience()


Se generaron 5000 nuevas ofertas en la tabla recruitment.workexperience.
